In [1]:
%config IPCompleter.greedy=True
%matplotlib notebook
import lib
lib.setup_logger('info', logfile='/home/adebahr/apercal/ipython-notebooks/commissioning/APERTIF/Bandpass_stability.log')
import apercal

root - INFO : Logging started!


root - INFO : To see the log in a bash window use the following command:


root - INFO : tail -n +1 -f /home/adebahr/apercal/ipython-notebooks/commissioning/APERTIF/Bandpass_stability.log


Setup logger with lib.setup_logger()


In [2]:
preflag = apercal.preflag('/home/adebahr/apercal/ipython-notebooks/commissioning/APERTIF/Bandpass_stability.cfg')

PREFLAG - INFO : ### Configuration file /home/adebahr/apercal/ipython-notebooks/commissioning/APERTIF/Bandpass_stability.cfg successfully read! ###


In [3]:
# This step gives an error in the end, but runs as intended. Have to fix this later
preflag.go()

PREFLAG - INFO : ### Starting pre-flagging of known flags ###


PREFLAG - INFO : # Moved to directory /data/adebahr/apertif/090808_3C286/raw #


PREFLAG - INFO : # Flagging auto-correlations for flux calibrator data #


PREFLAG - INFO : # Auto-correlation flagging for flux calibrator data done #


PREFLAG - INFO : # Flagging shadowed antennas for flux calibrator data #


...

PREFLAG - INFO : # Flagging timerange 19:45:0~21:00:0 for flux calibrator data #


PREFLAG - INFO : # Flagging of timerange 19:45:0~21:00:0 for flux calibrator data done #


PREFLAG - INFO : ### Pre-flagging of known flags done ###


PREFLAG - INFO : ### Doing pre-flagging with AOFlagger ###


TypeError: a float is required

In [4]:
convert = apercal.convert('/home/adebahr/apercal/ipython-notebooks/commissioning/APERTIF/Bandpass_stability.cfg')

CONVERT - INFO : ### Configuration file /home/adebahr/apercal/ipython-notebooks/commissioning/APERTIF/Bandpass_stability.cfg successfully read! ###


In [5]:
convert.go()

CONVERT - INFO : ########## FILE CONVERSION started ##########


CONVERT - INFO : ### Starting conversion from MS to UVFITS format ###


CONVERT - INFO : # Creating directory /data/adebahr/apertif/090808_3C286/crosscal #


CONVERT - INFO : # Moved to directory /data/adebahr/apertif/090808_3C286/crosscal #


CONVERT - INFO : ### Using CASA toolkit to convert from MS to UVFITS format! ###


...

CONVERT - INFO : ### Starting conversion from UVFITS to MIRIAD format ###


CONVERT - INFO : ### Using MIRIAD fits task to convert data from UVFITS to MIRIAD format ###


CONVERT - INFO : ### Converted UVFITS file 3C286.MS to MIRIAD format using MIRIAD task fits! ###


CONVERT - INFO : ### Conversion from UVFITS to MIRIAD format done! ###


CONVERT - INFO : ########## FILE CONVERSION done ##########


In [6]:
# Calibrate the flux calibrator now
# Initialise the crosscal module
ccal = apercal.ccal('/home/adebahr/apercal/ipython-notebooks/commissioning/APERTIF/Bandpass_stability.cfg')

CROSSCAL - INFO : ### Configuration file /home/adebahr/apercal/ipython-notebooks/commissioning/APERTIF/Bandpass_stability.cfg successfully read! ###


In [7]:
# Set the parameters for the bandpass calibration
gainint = '60'
gapint = '60'
bpint = '60'

# Execute the bandpass calibration
mfcal = lib.miriad('mfcal')
mfcal.vis = ccal.fluxcal
# Uncomment the next line if you additionally want to solve for delays
# mfcal.options = 'delay'
mfcal.stokes = 'XX'
mfcal.interval = gainint + ',' + gapint + ',' + bpint
mfcal.go()

['MfCal: $Revision: 1.20 $, $Date: 2017/05/08 02:31:10 $',
 'Reading the data ...',
 'Number correlations accepted: 11433931',
 'Number of frequency bands/settings: 1',
 'Number of polarisations selected: 1',
 'Number of solution intervals: 11',
 'Number of passband solution intervals: 11',
 'Initialising ...',
 'Using known frequency variation of 3c286',
 'Generating initial solution estimate ...',
 'Doing solution refinement ...',
 'Iter= 1, Solution Error:  1.314',
 'Iter= 2, Solution Error:  0.004',
 'Iter= 3, Solution Error:  0.003',
 'Iter= 4, Solution Error:  0.002',
 'Iter= 5, Solution Error:  0.001',
 'Iter= 6, Solution Error:  0.001',
 'Saving solution ...']

In [8]:
import sub_readmirlog
values, freqs = sub_readmirlog.get_bp(ccal.fluxcal)

In [9]:
# The bandpass results are ordered in a 3D numpy array now in the following order (antenna, frequency, solution interval)
print(values)

[[[ 0.61656  0.62122  0.61325 ...,  0.61607  0.6226   0.62194]
  [ 0.6349   0.63871  0.63726 ...,  0.63262  0.63669  0.64024]
  [ 0.70044  0.70101  0.68672 ...,  0.68706  0.7016   0.68585]
  ..., 
  [ 1.19099  1.18999  1.19587 ...,  1.18902  1.1781   1.17507]
  [ 1.08435  1.08751  1.08263 ...,  1.07953  1.08486  1.07575]
  [ 0.98957  0.98351  0.99244 ...,  0.9833   0.98545  0.99069]]

 [[ 0.6409   0.6394   0.64033 ...,  0.63669  0.64161  0.64021]
  [ 0.68173  0.68058  0.684   ...,  0.68665  0.6773   0.68561]
  [ 0.72599  0.72731  0.7384  ...,  0.73847  0.72294  0.74117]
  ..., 
  [ 1.10697  1.11135  1.11267 ...,  1.10588  1.10984  1.1073 ]
  [ 1.00618  1.02285  1.00765 ...,  1.00554  1.01064  1.01144]
  [ 0.93458  0.93082  0.92706 ...,  0.92222  0.92947  0.9275 ]]

 [[ 0.68074  0.66295  0.66088 ...,  0.63941  0.65699  0.67018]
  [ 0.68722  0.67871  0.66358 ...,  0.64268  0.66168  0.68146]
  [ 0.74471  0.73106  0.73689 ...,  0.70295  0.72822  0.74681]
  ..., 
  [ 1.12101  1.13198  1.120

In [10]:
# In addition it gives the a 1D-array of the frequency axis
print(freqs)

[ 1.307019   1.3070313  1.3070434 ...,  1.4413574  1.4413697  1.4413818]


In [11]:
# You can now easily slice and plot parts of the array in the usual manner
# If you want to plot the solutions for all timestamps of the first antenna
from matplotlib import pyplot as plt
plt.figure()
plt.plot(values[0,:,:])
plt.show()

<IPython.core.display.Javascript object>

<div id='e4bd384b-8c9e-45c7-8874-d563a040e4b4'></div>

In [12]:
# Or only for the first solution interval
from matplotlib import pyplot as plt
plt.figure()
plt.plot(values[0,:,0])
plt.show()

<IPython.core.display.Javascript object>

<div id='cd7b8d18-d703-4c9f-8371-fb247c2ff98d'></div>